# Parser Prototype

In [89]:
from lark import Lark

with open("grammar.lark") as grammar_file:
    grammar = grammar_file.read()

parser = Lark(grammar, start="start")

In [90]:
# vm, iface = get_consts(smtsorts, ["vm", "iface"])
# return And(
#     smtenc.element_class_fun(vm) == smtenc.classes["infrastructure_VirtualMachine"],
#     Not(
#         Exists(
#             [iface],
#             smtenc.association_rel(vm, smtenc.associations["infrastructure_ComputingNode::ifaces"], iface)
#         )
#     )
# )

example_expr_1 = r"""
    not vm is infrastructure.VirtualMachine
    and
    vm is not infrastructure.Storage
    iff
    not exists iface, apple (
        (vm has association infrastructure.ComputingNode->ifaces iface)
        or
        (vm has attribute infrastructure.ComputingNode->os Os1)
    )
"""

In [91]:
tree = parser.parse(example_expr_1)

print(tree.pretty())

start
  expression
    binary_op_exp
      double_implication
        expression
          binary_op_exp
            and_or_xor_exp
              negation
                not
                equal
                  class_or_const	vm
                  is
                  class_or_const	infrastructure.VirtualMachine
              and
              not_equal
                class_or_const	vm
                is not
                class_or_const	infrastructure.Storage
        iff
        negation
          not
          quantification
            exists
            consts
              iface
              apple
            expression
              binary_op_exp
                and_or_xor_exp
                  expression
                    association_expr
                      vm
                      has association
                      infrastructure.ComputingNode->ifaces
                      iface
                  or
                  expression
                    attribute_expr
 

We need the `ModelChecker` to import `SMTEncodings` and `SMTSorts` in order to create our Z3 constants programmatically.

Now the model checker should expose the *intermediate model checker* which should provide us with those two collections.

In [92]:
from mc_openapi.doml_mc import ModelChecker, DOMLVersion

# Import DOMLX as bytes
doml_document_path = "../../tests/doml/faas.domlx"
with open(doml_document_path, "rb") as xmif:
    doml_xmi = xmif.read()

model_checker = ModelChecker(doml_xmi, DOMLVersion.V2_0)

ENCODINGS =  model_checker.imc.smt_encoding
SORTS = model_checker.imc.smt_sorts

assert ENCODINGS and SORTS

I think we don't need to have the user declare each constant and value.
We can take care of that by saving the first occurence of each in a dictionary,
and subsequent uses of a variable will fetch the corresponding z3 ref from the dictionary.

In [93]:
from z3 import Const, ExprRef, SortRef
from mc_openapi.doml_mc.imc import SMTEncoding, SMTSorts

class UniqueVarStore:
    def __init__(self, smtenc: SMTEncoding, sort: SortRef):
        self.vars = dict()
        self.encoding = smtenc
        self.sort = sort
    
    def use(self, var_name: str) -> ExprRef:
        """Retrieves a variable with `var_name` from the store.
           If that variable doesn't exists, it is added to the store and returned.
        """
        if self.vars.get(var_name, None) is None:
            self.vars[var_name] = Const(var_name, self.sort)
        return self.vars[var_name]

    def keys(self) -> list[str]:
        return self.vars.keys()

    def clear(self):
        """Clears the store of all saved variables."""
        self.vars.clear()

constants_store = UniqueVarStore(ENCODINGS, SORTS.element_sort)
values_store = UniqueVarStore(ENCODINGS, SORTS.attr_data_sort)

RefHandler provides methods to create associations, classes, etc. to the Transformer without exposing Encodings and Sorts to it.

In [94]:
from z3 import DatatypeRef, FuncDeclRef

class RefHandler:
    def __init__(self, smtenc: SMTEncoding, smtsorts: SMTSorts):
        self.encoding = smtenc
        self.sorts = smtsorts
    
    def get_element_class(self, const: ExprRef) -> FuncDeclRef:
        return self.encoding.element_class_fun(const)

    def get_class(self, class_name: str) -> DatatypeRef:
        class_name = class_name.replace(".", "_")
        _class = self.encoding.classes.get(class_name, None)
        if _class is not None:
            return _class
        else:
            raise Exception(f"No class named '{class_name}' found.")
            # TODO: Try to suggest the correct class with difflib
            # see: https://docs.python.org/3/library/difflib.html?highlight=get_close_matches#difflib.get_close_matches

    def get_association(self, assoc_name: str) -> DatatypeRef:
        assoc_name = assoc_name.replace(".", "_")
        assoc_name = assoc_name.replace("->", "::")
        assoc = self.encoding.associations.get(assoc_name, None)
        if assoc is not None:
            return assoc
        else:
            raise Exception(f"No association named '{assoc_name}' found.")

    def get_association_rel(self, a: ExprRef, rel: DatatypeRef, b: ExprRef) -> DatatypeRef:
        return self.encoding.association_rel(a, rel, b)

    def get_attribute(self, attr_name: str) -> DatatypeRef:
        attr_name = attr_name.replace(".", "_")
        attr_name = attr_name.replace("->", "::")
        attr = self.encoding.attributes.get(attr_name, None)
        if attr is not None:
            return attr
        else:
            raise Exception(f"No attribute named '{attr_name}' found.")

    def get_attribute_rel(self, a: ExprRef, rel: DatatypeRef, b: ExprRef) -> DatatypeRef:
        return self.encoding.attribute_rel(a, rel, b)



ref_handler = RefHandler(ENCODINGS, SORTS)

# Test existent key
vm = ref_handler.get_class("infrastructure.VirtualMachine")
print("type: \t", type(vm))
print("value: \t", vm)
try:
    bad_class = ref_handler.get_class("infrastructure.Bicycle")
except Exception: 
    print("Exception catched!")

type: 	 <class 'z3.z3.DatatypeRef'>
value: 	 infrastructure_VirtualMachine
Exception catched!


This transformer will produce a Z3 expression to evaluate.

In [95]:
from z3 import Not, And, Or, Xor, Implies, Exists, ForAll

from lark import Transformer

constants_store.clear()
values_store.clear()

class DSLTransformer(Transformer):
    
    def start(self, args):
        return args[0]

    def expression(self, args):
        return args[0]

    def consts(self, args):
        return [constants_store.use(arg.value) for arg in args]

    def negation(self, args):
        return Not(args[1])

    def binary_op_exp(self, args):
        return args[0]
        
   
    def double_implication(self, args):        
        return args[0] == args[2]
    
    def implication(self, args):        
        return Implies(args[0], args[2])

    def and_or_xor_exp(self, args):        
        op = args[1].value
        a = args[0]
        b = args[2]

        if op == "and":
            return And(a, b)
        elif op == "or":
            return Or(a, b)
        else: # xor
            return Xor(a, b)

    def quantification(self, args):
        quantifier = args[0].value
       
        if quantifier == "exists":
            return Exists(args[1], args[2])
        else: # forall
            return ForAll(args[1], args[2])


    def association_expr(self, args):
        const1 = constants_store.use(args[0].value)
        assoc  = ref_handler.get_association(args[2].value)
        const2 = constants_store.use(args[3].value)
        
        return ref_handler.get_association_rel(const1, assoc, const2)

    def attribute_expr(self, args):
        const = constants_store.use(args[0].value)
        assoc  = ref_handler.get_attribute(args[2].value)
        value = values_store.use(args[3].value)
        return ref_handler.get_attribute_rel(const, assoc, value)

    def equal(self, args):
        return args[0] == args[2]

    def not_equal(self, args):
        return args[0] != args[2]

    def class_or_const(self, args):
        if args[0].type == "CONST":
            _const = constants_store.use(args[0].value)
            return ref_handler.get_element_class(_const)
        elif args[0].type == "CLASS":
            return ref_handler.get_class(args[0].value)


z3_parsed_exp = DSLTransformer().transform(tree)

z3_parsed_exp

And(Not(elem_class(vm) == infrastructure_VirtualMachine),
    elem_class(vm) != infrastructure_Storage) ==
Not(Exists([iface, apple],
           Or(association(vm,
                          infrastructure_ComputingNode::ifaces,
                          iface),
              attribute(vm,
                        infrastructure_ComputingNode::os,
                        Os1))))